In [ ]:
import pandas as pd  
import numpy as np  
import statsmodels.api as sm
from numpy import median
from numpy import mean

import os
for dirname, _, filenames in os.walk('/kaggle/input/kaggle-cirrhosis'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

from sklearn.model_selection import train_test_split 
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression 
from xgboost import XGBClassifier
import xgboost as xgb
from sklearn.metrics import log_loss, classification_report 
from sklearn.feature_selection import SelectKBest, f_regression
from sklearn.preprocessing import LabelEncoder
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import PowerTransformer
from sklearn.preprocessing import StandardScaler
from sklearn.impute import KNNImputer
from sklearn.model_selection import StratifiedKFold, cross_val_score
from sklearn.utils.class_weight import compute_class_weight
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import log_loss, classification_report
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier

import matplotlib.pyplot as plt
import seaborn as sns

# https://www.kaggle.com/code/abhirupghosh184098/cirrhosis-predictor-eda-ensemble-nn
# https://www.kaggle.com/code/ashishkumarak/ps3e26-liver-cirrhosis-eda-model

In [ ]:
train = pd.read_csv("/kaggle/input/kaggle-cirrhosis/train_df_clean.csv", 
                      delimiter = ",",
                      header = 0)

test = pd.read_csv("/kaggle/input/kaggle-cirrhosis/test.csv", 
                      delimiter = ",",
                      header = 0)

train.shape, test.shape # manque la colonne Status

## Partie Train

In [ ]:
train1 = train.copy()
train1.head()

In [ ]:
train1.info()

In [ ]:
for col in train1.select_dtypes(include=['object']).columns:
    unique_values = ', '.join(map(str, train1[col].unique()))
    print(f"Colonne '{col}': {unique_values}")

In [ ]:
(train1.isnull().sum() / len(train1) * 100).round(2)
#(train.isna().sum() / len(train) * 100).round(2)

### Colonnes Object (Mode)

In [ ]:
# mode pour les Hommes et Femmes selon la cible Status pour différentes colonnes (ancienne méthode)
# par la suite, Status remplacé par Stage

# DRUG:
# mode pour les Hommes selon la cible Status:
#train[train['Sex'] == 'M'].groupby('Status')['Drug'].agg(lambda x: x.mode().iloc[0])
# mode pour les Femmees selon la cible Status:
#train[train['Sex'] == 'F'].groupby('Status')['Drug'].agg(lambda x: x.mode().iloc[0])

# ASCITES:
# mode pour les Hommes selon la cible Status:
#train[train['Sex'] == 'M'].groupby('Status')['Ascites'].agg(lambda x: x.mode().iloc[0])
# mode pour les Femmes selon la cible Status:
#train[train['Sex'] == 'F'].groupby('Status')['Ascites'].agg(lambda x: x.mode().iloc[0])

# HEPATOMEGALY:
# mode pour les Hommes selon la cible Status:
#train[train['Sex'] == 'M'].groupby('Status')['Hepatomegaly'].agg(lambda x: x.mode().iloc[0])
# mode pour les Femmes selon la cible Status:
#train[train['Sex'] == 'F'].groupby('Status')['Hepatomegaly'].agg(lambda x: x.mode().iloc[0])

# SPIDERS:
# mode pour les Hommes selon la cible Status:
#train[train['Sex'] == 'M'].groupby('Status')['Spiders'].agg(lambda x: x.mode().iloc[0])
# mode pour les Femmes selon la cible Status:
#train[train['Sex'] == 'F'].groupby('Status')['Spiders'].agg(lambda x: x.mode().iloc[0])

#### Drug

In [ ]:
# mode pour les Hommes selon la cible Stage:
train1[train1['Sex'] == 'M'].groupby('Stage')['Drug'].agg(lambda x: x.mode().iloc[0])

In [ ]:
# mode pour les Femmees selon la cible Stage:
train1[train1['Sex'] == 'F'].groupby('Stage')['Drug'].agg(lambda x: x.mode().iloc[0])

#### Ascites

In [ ]:
# mode pour les Hommes selon la cible Stage:
train1[train1['Sex'] == 'M'].groupby('Stage')['Ascites'].agg(lambda x: x.mode().iloc[0])

In [ ]:
# mode pour les Femmes selon la cible Stage:
train1[train1['Sex'] == 'F'].groupby('Stage')['Ascites'].agg(lambda x: x.mode().iloc[0])


#### Hepatomegaly

In [ ]:
# mode pour les Hommes selon la cible Stage:
train1[train1['Sex'] == 'M'].groupby('Stage')['Hepatomegaly'].agg(lambda x: x.mode().iloc[0])

In [ ]:
# mode pour les Femmes selon la cible Stage:
train1[train1['Sex'] == 'F'].groupby('Stage')['Hepatomegaly'].agg(lambda x: x.mode().iloc[0])

#### Spiders

In [ ]:
# mode pour les Hommes selon la cible Stage:
train1[train1['Sex'] == 'M'].groupby('Stage')['Spiders'].agg(lambda x: x.mode().iloc[0])

In [ ]:
# mode pour les Femmes selon la cible Stage:
train1[train1['Sex'] == 'F'].groupby('Stage')['Spiders'].agg(lambda x: x.mode().iloc[0])

In [ ]:
# modes_drug = {
#     ('M', 'C'): 'Placebo',
#     ('M', 'CL'): 'D-penicillamine',
#     ('M', 'D'): 'D-penicillamine',
#     ('F', 'C'): 'D-penicillamine',
#     ('F', 'CL'): 'D-penicillamine',
#     ('F', 'D'): 'D-penicillamine'
# }

# modes_ascites = {
#     ('M', 'C'): 'N',
#     ('M', 'CL'): 'N',
#     ('M', 'D'): 'N',
#     ('F', 'C'): 'N',
#     ('F', 'CL'): 'N',
#     ('F', 'D'): 'N'
# }

# modes_hepatomegaly = {
#     ('M', 'C'): 'N',
#     ('M', 'CL'): 'Y',
#     ('M', 'D'): 'Y',
#     ('F', 'C'): 'N',
#     ('F', 'CL'): 'Y',
#     ('F', 'D'): 'Y'
# }

# modes_spiders = {
#     ('M', 'C'): 'N',
#     ('M', 'CL'): 'N',
#     ('M', 'D'): 'N',
#     ('F', 'C'): 'N',
#     ('F', 'CL'): 'N',
#     ('F', 'D'): 'N'
# }

# def na_par_mode(row):
#     key = (row['Sex'], row['Status'])
#     if pd.isna(row['Drug']):
#         row['Drug'] = modes_drug[key]
#     if pd.isna(row['Ascites']):
#         row['Ascites'] = modes_ascites[key]
#     if pd.isna(row['Hepatomegaly']):   

In [ ]:
# modes pour chaque colonne selon la modalité 'Stage'
modes_drug = {
    ('M', 1.0): 'Placebo',
    ('M', 2.0): 'D-penicillamine',
    ('M', 3.0): 'D-penicillamine',
    ('M', 4.0): 'D-penicillamine',
    ('F', 1.0): 'D-penicillamine',
    ('F', 2.0): 'D-penicillamine',
    ('F', 3.0): 'D-penicillamine',
    ('F', 4.0): 'Placebo'
}

modes_ascites = {
    ('M', 1.0): 'N',
    ('M', 2.0): 'N',
    ('M', 3.0): 'N',
    ('M', 4.0): 'N',
    ('F', 1.0): 'N',
    ('F', 2.0): 'N',
    ('F', 3.0): 'N',
    ('F', 4.0): 'N'
}

modes_hepatomegaly = {
    ('M', 1.0): 'N',
    ('M', 2.0): 'N',
    ('M', 3.0): 'N',
    ('M', 4.0): 'Y',
    ('F', 1.0): 'N',
    ('F', 2.0): 'N',
    ('F', 3.0): 'N',
    ('F', 4.0): 'Y'
}

modes_spiders = {
    ('M', 1.0): 'N',
    ('M', 2.0): 'N',
    ('M', 3.0): 'N',
    ('M', 4.0): 'N',
    ('F', 1.0): 'N',
    ('F', 2.0): 'N',
    ('F', 3.0): 'N',
    ('F', 4.0): 'N'
}

def na_par_mode(row):
    key = (row['Sex'], row['Stage'])
    if pd.isna(row['Drug']):
        row['Drug'] = modes_drug[key]
    if pd.isna(row['Ascites']):
        row['Ascites'] = modes_ascites[key]
    if pd.isna(row['Hepatomegaly']):
        row['Hepatomegaly'] = modes_hepatomegaly[key]
    if pd.isna(row['Spiders']):
        row['Spiders'] = modes_spiders[key]
    return row

train1 = train1.apply(na_par_mode, axis=1)

### Colonnes Numériques (Médiane)

In [ ]:
# la médiane pour chaque colonne numérique, par sexe et par modalité de Stage
medianes = train1.groupby(['Sex', 'Stage'])[['Cholesterol', 'Copper', 'Alk_Phos', 
                                             'SGOT', 'Tryglicerides', 'Platelets', 
                                             'Prothrombin']].median()

medianes

In [ ]:
moyennes = train1.groupby(['Sex', 'Stage'])[['Cholesterol', 'Copper', 'Alk_Phos', 
                                             'SGOT', 'Tryglicerides', 'Platelets', 
                                             'Prothrombin']].mean().round(2)
moyennes

In [ ]:
colonnes_a_traiter = ['Cholesterol', 'Copper', 'Alk_Phos', 
                       'SGOT', 'Tryglicerides', 'Platelets', 
                       'Prothrombin']

def na_par_mediane(row):
    for col in colonnes_a_traiter:
        if pd.isna(row[col]):
            row[col] = medianes.loc[(row['Sex'], row['Stage']), col]
    return row

train1 = train1.apply(na_par_mediane, axis=1)

In [ ]:
train1.info()

#### Normalisation avec PowerTransformer: pour rendre les données plus gaussiennes

In [ ]:
train2 = train1.copy()
# Normalisation avec PowerTransformer: pour rendre les données plus gaussienne
colonnes_to_normalize = ['Bilirubin', 'Cholesterol', 'Albumin', 'Copper', 'Alk_Phos', 
                        'SGOT', 'Tryglicerides', 'Platelets', 'Prothrombin']

transformer = PowerTransformer(method = 'yeo-johnson')
train2[colonnes_to_normalize] = transformer.fit_transform(train2[colonnes_to_normalize])

#### Encoding

In [ ]:
for col in train2.select_dtypes(include=['object']).columns:
    unique_values = ', '.join(map(str, train2[col].unique()))
    print(f"Colonne '{col}': {unique_values}")

In [ ]:
train2['Stage'][:3]

In [ ]:
# Custom Mapping pour Stage car l'ordre a l'air d'avoir son importance 
# This encoding is particularly useful for ordinal variable where the order of categories is important
# To make sure that the learning algorithm interprets the ordinal variables correctly, 
# we can map the categorical values to integer values manually. 
print(train2['Stage'].unique())  
train2['Stage'] = train2['Stage'].map({1.0: 0, 2.0: 1, 3.0: 2, 4.0: 3})
print(train2['Stage'].unique())

In [ ]:
# OneHot encoding pour les autres
# Liste des colonnes catégoriques
colonnes_cat = ['Drug', 'Sex', 'Ascites', 'Hepatomegaly', 'Spiders', 'Edema']

train_encoded = pd.get_dummies(train2, columns=colonnes_cat, drop_first=False)
train_encoded.head()

In [ ]:
train_encoded.columns

In [ ]:
# la colonne cible avec Label Encoding
label_encoder = LabelEncoder()
train_encoded['Status_encoded'] = label_encoder.fit_transform(train_encoded['Status']) 
train_encoded.drop('Status', axis=1, inplace=True)
train_encoded['Status_encoded'].value_counts()

In [ ]:
# le modèle 
X = train_encoded.drop(['id', 'Status_encoded'], axis=1) 
y = train_encoded['Status_encoded']  

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

model = LogisticRegression(multi_class='multinomial', solver='lbfgs', max_iter=10000)
# solver='lbfgs' par défaut

model.fit(X_train, y_train)

y_pred_proba = model.predict_proba(X_test)

logloss = log_loss(y_test, y_pred_proba)
print(f"Log Loss: {logloss}")

y_pred = model.predict(X_test)
print(classification_report(y_test, y_pred, target_names=['C', 'CL', 'D']))

### Autre méthode

In [ ]:
train.info()

In [ ]:
test['Drug'].replace('Drug', np.nan, inplace=True)
test['Spiders'].replace('C', np.nan, inplace=True)
for col in test.select_dtypes(include=['object']).columns:
    unique_values = ', '.join(map(str, test[col].unique()))
    print(f"Colonne '{col}': {unique_values}")

In [ ]:
test.info()

In [ ]:
train_drug = train.copy()
test_drug = test.copy()

# KNNImputer pour les colonnes numériques pour l'imputation
num_cols = ['Cholesterol', 'Copper', 'Alk_Phos', 'SGOT', 'Tryglicerides', 'Platelets', 'Prothrombin']
imputer = KNNImputer(n_neighbors=122)
imputer_test = KNNImputer(n_neighbors=100)

train_drug[num_cols] = imputer.fit_transform(train_drug[num_cols])
test_drug[num_cols] = imputer_test.fit_transform(test_drug[num_cols])

In [ ]:
train_drug['Drug_missing'] = train_drug['Drug'].isna().astype(int)

label_encoders = {}
categorical_cols = ['Drug', 'Sex', 'Ascites', 'Hepatomegaly', 'Spiders', 'Edema']

for col in categorical_cols:
    le = LabelEncoder()
    train_drug[col] = train_drug[col].astype(str)  
    train_drug[col] = le.fit_transform(train_drug[col])
    label_encoders[col] = le

not_null_data = train_drug[train_drug['Drug_missing'] == 0]
null_data = train_drug[train_drug['Drug_missing'] == 1]

if null_data.empty:
    print("Aucune valeur manquante pour 'Drug'.")
else:
    features = [col for col in train_drug.columns if col not in ['Drug', 'Drug_missing', 'Status'] and train_drug[col].isna().sum() == 0]
    
    X_train, X_test, y_train, y_test = train_test_split(
        not_null_data[features], not_null_data['Drug'], test_size=0.2, random_state=42
    )
    
    randfor = RandomForestClassifier(random_state=0, n_estimators=100, n_jobs=-1)
    param_grid = {
        'criterion': ['gini', 'entropy'],
        'max_depth': np.arange(1, 10)
    }
    gridmodel = GridSearchCV(randfor, param_grid, cv=5, scoring='accuracy', n_jobs=-1)
    gridmodel.fit(X_train, y_train)
    
    print("Meilleurs paramètres :", gridmodel.best_params_)
    best_model = gridmodel.best_estimator_
    
    test_predictions = best_model.predict(X_test)
    accuracy = round((test_predictions == y_test).sum() / len(test_predictions), 3)
    print(f"Précision sur l'ensemble de test : {accuracy}")

    predicted_values = best_model.predict(null_data[features])
    train_drug.loc[train_drug['Drug_missing'] == 1, 'Drug'] = predicted_values

train_drug['Drug'] = label_encoders['Drug'].inverse_transform(train_drug['Drug'].astype(int))

train_drug.drop('Drug_missing', axis=1, inplace=True)
train_drug[['Drug']].head(10)

In [ ]:
test_drug['Drug_missing'] = test_drug['Drug'].isna().astype(int)

label_encoders = {}
categorical_cols = ['Drug', 'Sex', 'Ascites', 'Hepatomegaly', 'Spiders', 'Edema']

for col in categorical_cols:
    le = LabelEncoder()
    test_drug[col] = test_drug[col].astype(str)  
    test_drug[col] = le.fit_transform(test_drug[col])
    label_encoders[col] = le

not_null_data = test_drug[test_drug['Drug_missing'] == 0]
null_data = test_drug[test_drug['Drug_missing'] == 1]

if null_data.empty:
    print("Aucune valeur manquante pour 'Drug'.")
else:
    features = [col for col in test_drug.columns if col not in ['Drug', 'Drug_missing', 'Status'] and test_drug[col].isna().sum() == 0]
    
    X_train, X_test, y_train, y_test = train_test_split(
        not_null_data[features], not_null_data['Drug'], test_size=0.2, random_state=42
    )
    
    randfor = RandomForestClassifier(random_state=0, n_estimators=100, n_jobs=-1)
    param_grid = {
        'criterion': ['gini', 'entropy'],
        'max_depth': np.arange(1, 10)
    }
    gridmodel = GridSearchCV(randfor, param_grid, cv=5, scoring='accuracy', n_jobs=-1)
    gridmodel.fit(X_train, y_train)
    
    print("Meilleurs paramètres :", gridmodel.best_params_)
    best_model = gridmodel.best_estimator_
    
    test_predictions = best_model.predict(X_test)
    accuracy = round((test_predictions == y_test).sum() / len(test_predictions), 3)
    print(f"Précision sur l'ensemble de test : {accuracy}")

    predicted_values = best_model.predict(null_data[features])
    test_drug.loc[test_drug['Drug_missing'] == 1, 'Drug'] = predicted_values

test_drug['Drug'] = label_encoders['Drug'].inverse_transform(test_drug['Drug'].astype(int))

test_drug.drop('Drug_missing', axis=1, inplace=True)
test_drug[['Drug']].head(10)

In [ ]:
train_ascites = train.copy()
test_ascites = test.copy()

# KNNImputer pour les colonnes numériques pour l'imputation
num_cols = ['Cholesterol', 'Copper', 'Alk_Phos', 'SGOT', 'Tryglicerides', 'Platelets', 'Prothrombin']
imputer = KNNImputer(n_neighbors=122)

train_ascites[num_cols] = imputer.fit_transform(train_ascites[num_cols])
test_ascites[num_cols] = imputer_test.fit_transform(test_ascites[num_cols])
train_ascites.info()

In [ ]:
train_ascites['Ascites_missing'] = train_ascites['Ascites'].isna().astype(int)

label_encoders = {}
categorical_cols = ['Drug', 'Sex', 'Ascites', 'Hepatomegaly', 'Spiders', 'Edema']

for col in categorical_cols:
    le = LabelEncoder()
    train_ascites[col] = train_ascites[col].astype(str)  
    train_ascites[col] = le.fit_transform(train_ascites[col])
    label_encoders[col] = le

not_null_data = train_ascites[train_ascites['Ascites_missing'] == 0]
null_data = train_ascites[train_ascites['Ascites_missing'] == 1]

if null_data.empty:
    print("Aucune valeur manquante pour 'Ascites'.")
else:
    features = [
        col for col in train_ascites.columns 
        if col not in ['Ascites', 'Ascites_missing', 'Status'] 
           and train_ascites[col].isna().sum() == 0
    ]
    
    X_train, X_test, y_train, y_test = train_test_split(
        not_null_data[features], 
        not_null_data['Ascites'], 
        test_size=0.2, 
        random_state=42
    )
    
    randfor = RandomForestClassifier(random_state=0, n_estimators=100, n_jobs=-1)
    param_grid = {
        'criterion': ['gini', 'entropy'],
        'max_depth': np.arange(1, 10)
    }
    gridmodel = GridSearchCV(randfor, param_grid, cv=5, scoring='accuracy', n_jobs=-1)
    gridmodel.fit(X_train, y_train)
    
    print("Meilleurs paramètres :", gridmodel.best_params_)
    best_model = gridmodel.best_estimator_
    
    test_predictions = best_model.predict(X_test)
    accuracy = round((test_predictions == y_test).sum() / len(test_predictions), 3)
    print(f"Précision sur l'ensemble de test : {accuracy}")

    predicted_values = best_model.predict(null_data[features])
    train_ascites.loc[train_ascites['Ascites_missing'] == 1, 'Ascites'] = predicted_values

train_ascites['Ascites'] = label_encoders['Ascites'].inverse_transform(
    train_ascites['Ascites'].astype(int)
)

train_ascites.drop('Ascites_missing', axis=1, inplace=True)
train_ascites[['Ascites']].head(10)

In [ ]:
test_ascites['Ascites_missing'] = test_ascites['Ascites'].isna().astype(int)

label_encoders = {}
categorical_cols = ['Drug', 'Sex', 'Ascites', 'Hepatomegaly', 'Spiders', 'Edema']

for col in categorical_cols:
    le = LabelEncoder()
    test_ascites[col] = test_ascites[col].astype(str)  
    test_ascites[col] = le.fit_transform(test_ascites[col])
    label_encoders[col] = le

not_null_data = test_ascites[test_ascites['Ascites_missing'] == 0]
null_data = test_ascites[test_ascites['Ascites_missing'] == 1]

if null_data.empty:
    print("Aucune valeur manquante pour 'Ascites'.")
else:
    features = [
        col for col in test_ascites.columns 
        if col not in ['Ascites', 'Ascites_missing', 'Status'] 
           and test_ascites[col].isna().sum() == 0
    ]
    
    X_train, X_test, y_train, y_test = train_test_split(
        not_null_data[features], 
        not_null_data['Ascites'], 
        test_size=0.2, 
        random_state=42
    )
    
    randfor = RandomForestClassifier(random_state=0, n_estimators=100, n_jobs=-1)
    param_grid = {
        'criterion': ['gini', 'entropy'],
        'max_depth': np.arange(1, 10)
    }
    gridmodel = GridSearchCV(randfor, param_grid, cv=5, scoring='accuracy', n_jobs=-1)
    gridmodel.fit(X_train, y_train)
    
    print("Meilleurs paramètres :", gridmodel.best_params_)
    best_model = gridmodel.best_estimator_
    
    test_predictions = best_model.predict(X_test)
    accuracy = round((test_predictions == y_test).sum() / len(test_predictions), 3)
    print(f"Précision sur l'ensemble de test : {accuracy}")

    predicted_values = best_model.predict(null_data[features])
    test_ascites.loc[test_ascites['Ascites_missing'] == 1, 'Ascites'] = predicted_values

test_ascites['Ascites'] = label_encoders['Ascites'].inverse_transform(
    test_ascites['Ascites'].astype(int)
)

test_ascites.drop('Ascites_missing', axis=1, inplace=True)

test_ascites[['Ascites']].head(10)

In [ ]:
train_hepatomegaly = train.copy()
test_hepatomegaly = test.copy()

# KNNImputer pour les colonnes numériques pour l'imputation
num_cols = ['Cholesterol', 'Copper', 'Alk_Phos', 'SGOT', 'Tryglicerides', 'Platelets', 'Prothrombin']
imputer = KNNImputer(n_neighbors=122)

train_hepatomegaly[num_cols] = imputer.fit_transform(train_hepatomegaly[num_cols])
test_hepatomegaly[num_cols] = imputer_test.fit_transform(test_hepatomegaly[num_cols])

train_hepatomegaly.info()

In [ ]:
train_hepatomegaly['Hepatomegaly_missing'] = train_hepatomegaly['Hepatomegaly'].isna().astype(int)

label_encoders = {}
categorical_cols = ['Drug', 'Sex', 'Ascites', 'Hepatomegaly', 'Spiders', 'Edema']

for col in categorical_cols:
    le = LabelEncoder()
    train_hepatomegaly[col] = train_hepatomegaly[col].astype(str)
    train_hepatomegaly[col] = le.fit_transform(train_hepatomegaly[col])
    label_encoders[col] = le

not_null_data = train_hepatomegaly[train_hepatomegaly['Hepatomegaly_missing'] == 0]
null_data = train_hepatomegaly[train_hepatomegaly['Hepatomegaly_missing'] == 1]

if null_data.empty:
    print("Aucune valeur manquante pour 'Hepatomegaly'.")
else:
    features = [
        col for col in train_hepatomegaly.columns
        if col not in ['Hepatomegaly', 'Hepatomegaly_missing', 'Status']
           and train_hepatomegaly[col].isna().sum() == 0
    ]
    
    X_train, X_test, y_train, y_test = train_test_split(
        not_null_data[features],
        not_null_data['Hepatomegaly'],
        test_size=0.2,
        random_state=42
    )
    
    randfor = RandomForestClassifier(random_state=0, n_estimators=100, n_jobs=-1)
    param_grid = {
        'criterion': ['gini', 'entropy'],
        'max_depth': np.arange(1, 10)
    }
    gridmodel = GridSearchCV(randfor, param_grid, cv=5, scoring='accuracy', n_jobs=-1)
    gridmodel.fit(X_train, y_train)
    
    print("Meilleurs paramètres :", gridmodel.best_params_)
    best_model = gridmodel.best_estimator_
    
    test_predictions = best_model.predict(X_test)
    accuracy = round((test_predictions == y_test).sum() / len(test_predictions), 3)
    print(f"Précision sur l'ensemble de test : {accuracy}")

    predicted_values = best_model.predict(null_data[features])
    train_hepatomegaly.loc[train_hepatomegaly['Hepatomegaly_missing'] == 1, 'Hepatomegaly'] = predicted_values

train_hepatomegaly['Hepatomegaly'] = label_encoders['Hepatomegaly'].inverse_transform(
    train_hepatomegaly['Hepatomegaly'].astype(int)
)

train_hepatomegaly.drop('Hepatomegaly_missing', axis=1, inplace=True)
train_hepatomegaly[['Hepatomegaly']].head(10)

In [ ]:
test_hepatomegaly['Hepatomegaly_missing'] = test_hepatomegaly['Hepatomegaly'].isna().astype(int)

label_encoders = {}
categorical_cols = ['Drug', 'Sex', 'Ascites', 'Hepatomegaly', 'Spiders', 'Edema']

for col in categorical_cols:
    le = LabelEncoder()
    test_hepatomegaly[col] = test_hepatomegaly[col].astype(str)
    test_hepatomegaly[col] = le.fit_transform(test_hepatomegaly[col])
    label_encoders[col] = le

not_null_data = test_hepatomegaly[test_hepatomegaly['Hepatomegaly_missing'] == 0]
null_data = test_hepatomegaly[test_hepatomegaly['Hepatomegaly_missing'] == 1]

if null_data.empty:
    print("Aucune valeur manquante pour 'Hepatomegaly'.")
else:
    features = [
        col for col in test_hepatomegaly.columns
        if col not in ['Hepatomegaly', 'Hepatomegaly_missing', 'Status']
           and test_hepatomegaly[col].isna().sum() == 0
    ]
    
    X_train, X_test, y_train, y_test = train_test_split(
        not_null_data[features],
        not_null_data['Hepatomegaly'],
        test_size=0.2,
        random_state=42
    )
    
    randfor = RandomForestClassifier(random_state=0, n_estimators=100, n_jobs=-1)
    param_grid = {
        'criterion': ['gini', 'entropy'],
        'max_depth': np.arange(1, 10)
    }
    gridmodel = GridSearchCV(randfor, param_grid, cv=5, scoring='accuracy', n_jobs=-1)
    gridmodel.fit(X_train, y_train)
    
    print("Meilleurs paramètres :", gridmodel.best_params_)
    best_model = gridmodel.best_estimator_
    
    test_predictions = best_model.predict(X_test)
    accuracy = round((test_predictions == y_test).sum() / len(test_predictions), 3)
    print(f"Précision sur l'ensemble de test : {accuracy}")

    predicted_values = best_model.predict(null_data[features])
    test_hepatomegaly.loc[test_hepatomegaly['Hepatomegaly_missing'] == 1, 'Hepatomegaly'] = predicted_values

test_hepatomegaly['Hepatomegaly'] = label_encoders['Hepatomegaly'].inverse_transform(
    test_hepatomegaly['Hepatomegaly'].astype(int)
)

test_hepatomegaly.drop('Hepatomegaly_missing', axis=1, inplace=True)
test_hepatomegaly[['Hepatomegaly']].head(10)

In [ ]:
train_spiders = train.copy()
test_spiders = test.copy()

# KNNImputer pour les colonnes numériques pour l'imputation
num_cols = ['Cholesterol', 'Copper', 'Alk_Phos', 'SGOT', 'Tryglicerides', 'Platelets', 'Prothrombin']
imputer = KNNImputer(n_neighbors=122)

train_spiders[num_cols] = imputer.fit_transform(train_spiders[num_cols])
test_spiders[num_cols] = imputer_test.fit_transform(test_spiders[num_cols])
train_spiders.info()

In [ ]:
train_spiders['Spiders_missing'] = train_spiders['Spiders'].isna().astype(int)

label_encoders = {}
categorical_cols = ['Drug', 'Sex', 'Ascites', 'Hepatomegaly', 'Spiders', 'Edema']

for col in categorical_cols:
    le = LabelEncoder()
    train_spiders[col] = train_spiders[col].astype(str)
    train_spiders[col] = le.fit_transform(train_spiders[col])
    label_encoders[col] = le

not_null_data = train_spiders[train_spiders['Spiders_missing'] == 0]
null_data = train_spiders[train_spiders['Spiders_missing'] == 1]

if null_data.empty:
    print("Aucune valeur manquante pour 'Spiders'.")
else:
    features = [
        col for col in train_spiders.columns
        if col not in ['Spiders', 'Spiders_missing', 'Status']
           and train_spiders[col].isna().sum() == 0
    ]
    
    X_train, X_test, y_train, y_test = train_test_split(
        not_null_data[features],
        not_null_data['Spiders'],
        test_size=0.2,
        random_state=42
    )
    
    randfor = RandomForestClassifier(random_state=0, n_estimators=100, n_jobs=-1)
    param_grid = {
        'criterion': ['gini', 'entropy'],
        'max_depth': np.arange(1, 10)
    }
    gridmodel = GridSearchCV(randfor, param_grid, cv=5, scoring='accuracy', n_jobs=-1)
    gridmodel.fit(X_train, y_train)
    
    print("Meilleurs paramètres :", gridmodel.best_params_)
    best_model = gridmodel.best_estimator_
    
    test_predictions = best_model.predict(X_test)
    accuracy = round((test_predictions == y_test).sum() / len(test_predictions), 3)
    print(f"Précision sur l'ensemble de test : {accuracy}")

    predicted_values = best_model.predict(null_data[features])
    train_spiders.loc[train_spiders['Spiders_missing'] == 1, 'Spiders'] = predicted_values

train_spiders['Spiders'] = label_encoders['Spiders'].inverse_transform(
    train_spiders['Spiders'].astype(int)
)

train_spiders.drop('Spiders_missing', axis=1, inplace=True)
train_spiders[['Spiders']].head(10)

In [ ]:
test_spiders['Spiders_missing'] = test_spiders['Spiders'].isna().astype(int)

label_encoders = {}
categorical_cols = ['Drug', 'Sex', 'Ascites', 'Hepatomegaly', 'Spiders', 'Edema']

for col in categorical_cols:
    le = LabelEncoder()
    test_spiders[col] = test_spiders[col].astype(str)
    test_spiders[col] = le.fit_transform(test_spiders[col])
    label_encoders[col] = le

not_null_data = test_spiders[test_spiders['Spiders_missing'] == 0]
null_data = test_spiders[test_spiders['Spiders_missing'] == 1]

if null_data.empty:
    print("Aucune valeur manquante pour 'Spiders'.")
else:
    features = [
        col for col in test_spiders.columns
        if col not in ['Spiders', 'Spiders_missing', 'Status']
           and test_spiders[col].isna().sum() == 0
    ]
    
    X_train, X_test, y_train, y_test = train_test_split(
        not_null_data[features],
        not_null_data['Spiders'],
        test_size=0.2,
        random_state=42
    )
    
    randfor = RandomForestClassifier(random_state=0, n_estimators=100, n_jobs=-1)
    param_grid = {
        'criterion': ['gini', 'entropy'],
        'max_depth': np.arange(1, 10)
    }
    gridmodel = GridSearchCV(randfor, param_grid, cv=5, scoring='accuracy', n_jobs=-1)
    gridmodel.fit(X_train, y_train)
    
    print("Meilleurs paramètres :", gridmodel.best_params_)
    best_model = gridmodel.best_estimator_
    
    test_predictions = best_model.predict(X_test)
    accuracy = round((test_predictions == y_test).sum() / len(test_predictions), 3)
    print(f"Précision sur l'ensemble de test : {accuracy}")

    predicted_values = best_model.predict(null_data[features])
    test_spiders.loc[test_spiders['Spiders_missing'] == 1, 'Spiders'] = predicted_values

test_spiders['Spiders'] = label_encoders['Spiders'].inverse_transform(
    test_spiders['Spiders'].astype(int)
)

test_spiders.drop('Spiders_missing', axis=1, inplace=True)
test_spiders[['Spiders']].head(10)

In [ ]:
id_drug_df = train_drug[['id', 'Drug']]
id_ascites_df = train_ascites[['id', 'Ascites']]
id_hepatomegaly_df = train_hepatomegaly[['id', 'Hepatomegaly']]
id_spiders_df = train_spiders[['id', 'Spiders']]

id_drug_df_test = test_drug[['id', 'Drug']]
id_ascites_df_test = test_ascites[['id', 'Ascites']]
id_hepatomegaly_df_test = test_hepatomegaly[['id', 'Hepatomegaly']]
id_spiders_df_test = test_spiders[['id', 'Spiders']]

In [ ]:
train_clean = train.copy()
test_clean = test.copy()

# KNNImputer pour les colonnes numériques pour l'imputation
num_cols = ['Cholesterol', 'Copper', 'Alk_Phos', 'SGOT', 'Tryglicerides', 'Platelets', 'Prothrombin']
imputer = KNNImputer(n_neighbors=122)

train_clean[num_cols] = imputer.fit_transform(train_clean[num_cols])
test_clean[num_cols] = imputer_test.fit_transform(test_clean[num_cols])
train_clean.info()

In [ ]:
columns_to_drop = ['Drug', 'Ascites', 'Hepatomegaly', 'Spiders']
train_clean = train_clean.drop(columns=columns_to_drop, errors='ignore')
test_clean = test_clean.drop(columns=columns_to_drop, errors='ignore')

In [ ]:
train_clean = train_clean.merge(id_drug_df, on='id', how='left')
train_clean = train_clean.merge(id_ascites_df, on='id', how='left')
train_clean = train_clean.merge(id_hepatomegaly_df, on='id', how='left')
train_clean = train_clean.merge(id_spiders_df, on='id', how='left')

test_clean = test_clean.merge(id_drug_df_test, on='id', how='left')
test_clean = test_clean.merge(id_ascites_df_test, on='id', how='left')
test_clean = test_clean.merge(id_hepatomegaly_df_test, on='id', how='left')
test_clean = test_clean.merge(id_spiders_df_test, on='id', how='left')

In [ ]:
train_clean.head(10)

In [ ]:
test_clean.info()

In [ ]:
test_clean.head(10)
test_clean['N_Days'].fillna(test_clean['N_Days'].median(skipna=True), inplace=True)

In [ ]:
test_clean.info()

In [ ]:
train_clean.info()

In [ ]:
train_clean_copy = train_clean.copy()
test_clean_copy = test_clean.copy()

# Normalisation avec PowerTransformer: pour rendre les données plus gaussienne
colonnes_to_normalize = ['Bilirubin', 'Cholesterol', 'Albumin', 'Copper', 'Alk_Phos', 
                        'SGOT', 'Tryglicerides', 'Platelets', 'Prothrombin']

transformer = PowerTransformer(method = 'yeo-johnson')
train_clean_copy[colonnes_to_normalize] = transformer.fit_transform(train_clean_copy[colonnes_to_normalize])
test_clean_copy[colonnes_to_normalize] = transformer.fit_transform(test_clean_copy[colonnes_to_normalize])

In [ ]:
for col in train_clean_copy.select_dtypes(include=['object']).columns:
    unique_values = ', '.join(map(str, train_clean_copy[col].unique()))
    print(f"Colonne '{col}': {unique_values}")

In [ ]:
train_clean_copy['Stage'][:3]

In [ ]:
# Custom Mapping pour Stage car l'ordre a l'air d'avoir son importance 
# This encoding is particularly useful for ordinal variable where the order of categories is important
# To make sure that the learning algorithm interprets the ordinal variables correctly, 
# we can map the categorical values to integer values manually. 
print(train_clean_copy['Stage'].unique())  
train_clean_copy['Stage'] = train_clean_copy['Stage'].map({1.0: 0, 2.0: 1, 3.0: 2, 4.0: 3})
test_clean_copy['Stage'] = test_clean_copy['Stage'].map({1.0: 0, 2.0: 1, 3.0: 2, 4.0: 3})
print(train_clean_copy['Stage'].unique())

In [ ]:
# OneHot encoding pour les autres
# Liste des colonnes catégoriques
colonnes_cat = ['Drug', 'Sex', 'Ascites', 'Hepatomegaly', 'Spiders', 'Edema']

train_clean_copy_encoded = pd.get_dummies(train_clean_copy, columns=colonnes_cat, drop_first=False)
test_clean_copy_encoded = pd.get_dummies(test_clean_copy, columns=colonnes_cat, drop_first=False)
train_clean_copy_encoded.head()

In [ ]:
train_clean_copy_encoded.columns

In [ ]:
# la colonne cible avec Label Encoding
label_encoder = LabelEncoder()
train_clean_copy_encoded['Status_encoded'] = label_encoder.fit_transform(train_clean_copy_encoded['Status']) 
train_clean_copy_encoded.drop('Status', axis=1, inplace=True)
train_clean_copy_encoded['Status_encoded'].value_counts()

In [ ]:
# le modèle 
X = train_clean_copy_encoded.drop(['id', 'Status_encoded'], axis=1) 
y = train_clean_copy_encoded['Status_encoded']  

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

model = LogisticRegression(multi_class='multinomial', solver='lbfgs', max_iter=10000)
# solver='lbfgs' par défaut

model.fit(X_train, y_train)

y_pred_proba = model.predict_proba(X_test)

logloss = log_loss(y_test, y_pred_proba)
print(f"Log Loss: {logloss}")

y_pred = model.predict(X_test)
print(classification_report(y_test, y_pred, target_names=['C', 'CL', 'D']))

In [ ]:
train_clean_copy_encoded

In [ ]:
X = train_clean_copy_encoded.drop(['id', 'Status_encoded'], axis=1)
y = train_clean_copy_encoded['Status_encoded']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42)

xgb_model = xgb.XGBClassifier(
    objective='multi:softprob', # Pour prédire les probabilités pour chaque classe
    num_class=3,                # Nombre de classes (C, CL, D)
    eval_metric='mlogloss',     # Log-loss pour l'évaluation
    use_label_encoder=False,    # Pour éviter les avertissements (à partir de XGBoost v1.3.0)
    max_depth=6,                # Profondeur de l'arbre (à ajuster selon les données)
    learning_rate=0.1,          # Taux d'apprentissage
    n_estimators=100,           # Nombre d'arbres (à ajuster pour éviter le sur-apprentissage)
    subsample=0.8,              # Fraction d'échantillons utilisée pour chaque arbre
    colsample_bytree=0.8,       # Fraction des colonnes utilisées pour chaque arbre
    seed=42                     # Pour la reproductibilité
)

xgb_model.fit(X_train, y_train)

y_pred_proba = xgb_model.predict_proba(X_test)

logloss = log_loss(y_test, y_pred_proba)
print(f"Log Loss avec XGBoost : {logloss:.4f}")

y_pred = xgb_model.predict(X_test)

print(classification_report(y_test, y_pred, target_names=['C', 'CL', 'D']))

In [ ]:
X_test_final = test_clean_copy_encoded.drop(['id'], axis=1)

In [ ]:
X.columns

In [ ]:
X_test_final.columns

In [ ]:
y_test_proba = xgb_model.predict_proba(X_test_final)

df_results = test_clean_copy_encoded[['id']].copy()
df_results['Status_C'] = y_test_proba[:, 0]
df_results['Status_CL'] = y_test_proba[:, 1]
df_results['Status_D'] = y_test_proba[:, 2]

df_results.head(10)